In [ ]:
%run nb_temp_load_cfg

In [ ]:
# Kill above cell

In [ ]:
# PL: The API function "fn_api_request()" is "system", because it is also used in PowerBI Refresh

# System

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_bronze_function_extract_api - Start ----------\n")

In [ ]:
medallion_name = "Bronze"
if is_debug: print(f"medallion_name: {medallion_name}")

In [ ]:
# Current session info
if is_debug: display(ss.getActiveSession())

# Function

## Add initial values in table "lh_cfg.extract_parameter_api"

In [ ]:
if "API" in list_technology:
    def fn_add_initial_extract_parameter_api(
        technology
        , frequency
    ):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            # Edit this query to cover your "datetime_from_to" filters
            sql_code = f"""SELECT
    "{technology}"              AS `technology`
    , "{frequency}"             AS `frequency`
    , EO.`table_name`
    , EO.`base_url`
    , "datetime_from"           AS `name`
    , "1900-01-01 00:00:00.000" AS `value`
FROM
    delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_api`              AS EO
    LEFT JOIN delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter_api` AS EP
        ON EO.`technology`              = EP.`technology`
        AND EO.`frequency`              = EP.`frequency`
        AND IFNULL(EO.`table_name`, "") = IFNULL(EP.`table_name`, "")
        AND IFNULL(EO.`base_url`, "")   = IFNULL(EP.`base_url`, "")
WHERE
    EP.`technology`       IS NULL
    AND EO.`technology`   = "{technology}"
    AND EO.`frequency`    = "{frequency}"
    AND EO.`is_extracted` = 1
    AND
    (
        EO.`body`      LIKE "%^datetime_from^%"
        OR EO.`params` LIKE "%^datetime_from^%"
    )

UNION ALL SELECT
    "{technology}"            AS `technology`
    , "{frequency}"           AS `frequency`
    , EO.`table_name`
    , EO.`base_url`
    , "datetime_to"       AS `name`
    , NULL                AS `value`
FROM
    delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_api`              AS EO
    LEFT JOIN delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter_api` AS EP
        ON EO.`technology`            = EP.`technology`
        AND EO.`frequency`            = EP.`frequency`
        AND IFNULL(EO.`table_name`, "") = IFNULL(EP.`table_name`, "")
        AND IFNULL(EO.`base_url`, "")   = IFNULL(EP.`base_url`, "")
WHERE
    EP.`technology`       IS NULL
    AND EO.`technology`   = "{technology}"
    AND EO.`frequency`    = "{frequency}"
    AND EO.`is_extracted` = 1
    AND
    (
        EO.`body`      LIKE "%^datetime_to^%"
        OR EO.`params` LIKE "%^datetime_to^%"
    )
ORDER BY
    `technology`
    , `frequency`
    , `table_name`
    , `base_url`;"""

            rv_initial_extract_parameter = fn_execute_spark_sql(sql_code)
            sdf                          = rv_initial_extract_parameter[2]
            sdf_count                    = rv_initial_extract_parameter[3]

            if sdf_count > 0:
                # PL: 
                # Debug this - Start
                # Bug: Uses another sdf (the one from the "Test" section of fn_save_sdf_as_table())
                #fn_save_sdf_as_table("sdf", \
                #    "delta", \
                #    "append", \
                #    f"{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter", \
                #    None)

                sdf.write\
                    .format("delta")\
                    .mode("append")\
                    .save(f"{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter_api")
                # Debug this - End

            alert             = "Success"
            alert_description = f"New rows (count): {str(sdf_count)}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, sdf_count)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
                fn_print_debug_info(alert, fn_name, par)
                del par

            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

In [ ]:
technology = "API"
frequency = "Daily"

fn_add_initial_extract_parameter_api(technology, frequency)

In [ ]:
display(global_parameter)

In [ ]:
# if "API" in list_technology:
#     def fn_set_headers(
#         headers
#         , authorization_type
#         , authorization_value
#         , response_format
#     ):
#         fn_name        = stk()[0][3]
#         caller_fn_name = stk()[1].function.replace("<module>", "")
#         if is_debug: par = {}

#         try:

headers = {"User-Agent": "zPL Concept"}
authorization_type = "None"
authorization_value = ""
response_format = "JSON"

content_type = f"application/{response_format.lower()}"
headers["Content-Type"] = content_type

#Lookup placeholders in table lh_cfg.dbo.global_parameter (*parameter_name*)
#Lookup placeholders in table lh_cfg.dbo.extract_parameter (^parameter_name^)

#if authorization_type == "Bearer":
    # read secret from KV
#    Parameter --> {"key": "*kv_test_key*"}
    global_parameter.kv*test_key
    secret = notebookutils.credentials.getSecret(global_parameter.keyvault_url, dict_parameter["keyvault_secret_name_user"])

print(headers)

In [ ]:
# Get Secret from Key-Vault
dict_parameter = {}
dict_parameter["keyvault_url"] = "https://kv-keyvault.vault.azure.net/"
dict_parameter["keyvault_secret_name_user"] = "sql-server-database-usr"
dict_parameter["keyvault_secret_name_password"] = "sql-server-database-pwd/"

username = notebookutils.credentials.getSecret(dict_parameter["keyvault_url"], dict_parameter["keyvault_secret_name_user"])
password = notebookutils.credentials.getSecret(dict_parameter["keyvault_url"], dict_parameter["keyvault_secret_name_password"])

print(username)
print(password)

# Unmask secrets
plain_username = str('')
for c in username:
    plain_username += str(print(c, end = ' ')).strip("None")
print(plain_username)

plain_password = str('')
for c in password:
    plain_password += str(print(c, end = ' ')).strip("None")
print(plain_password)

In [ ]:
# if "API" in list_technology:
#     def fn_set_params(
#         params
#     ):
#         fn_name        = stk()[0][3]
#         caller_fn_name = stk()[1].function.replace("<module>", "")
#         if is_debug: par = {}

#         try:

In [ ]:
# if "API" in list_technology:
#     def fn_set_body(
#         params
#     ):
#         fn_name        = stk()[0][3]
#         caller_fn_name = stk()[1].function.replace("<module>", "")
#         if is_debug: par = {}

#         try:

Lookup placeholders in table lh_cfg.dbo.extract_parameter_api (^parameter_name^)

In [ ]:
pdf_log.reset_index(drop = True, inplace = True)
display(pdf_log.sort_values(by = "logged_datetime", ascending = False))